In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms

import numpy as np
import os
import csv

In [2]:
BATCH_SIZE = 8
NUM_EPOCHS = 10

path = 'sjtu-m3dv-medical-3d-voxel-classification'
train_path = path+'/train_val'
test_path = path+'/test'

In [3]:
# 打开训练集结果文件
train_res = []
with open(path+'/train_val.csv') as f:
    next(f)
    f_csv = csv.reader(f)
    for row in f_csv:
        train_res.append(int(row[1]))
f.close()
# 打开训练集数据文件
train_npz = os.listdir(train_path)
train_num = len(train_npz)
train_voxel = []
train_seg = []
for i in range(train_num):
#for i in range(20):
    file = np.load(train_path+'/'+train_npz[i])
    voxel = file['voxel'].astype(np.float)
    seg = file['seg'].astype(np.float)
    train_voxel.append((voxel,train_res[i]))
    train_seg.append((seg*300,train_res[i]))
# 加载训练集
train_voxel_loader = data.DataLoader(train_voxel,
                                     batch_size=BATCH_SIZE,
                                     shuffle=True,
                                     drop_last=True) 
train_seg_loader = data.DataLoader(train_seg,
                                   batch_size=BATCH_SIZE,
                                   shuffle=True,
                                   drop_last=True) 

In [3]:
# 打开测试集文件
test_npz = os.listdir(test_path)
test_num = len(test_npz)
test_voxel = []
test_seg = []
for i in range(test_num):
    file = np.load(test_path+'/'+test_npz[i])
    voxel = file['voxel'].astype(np.float)
    seg = file['seg'].astype(np.float)*300
    test_voxel.append(voxel)
    test_seg.append(seg)
# 加载测试集
test_voxel_loader = data.DataLoader(test_voxel,
                                    batch_size=BATCH_SIZE,
                                    shuffle=True,
                                    drop_last=True) 
test_seg_loader = data.DataLoader(test_seg,
                                  batch_size=BATCH_SIZE,
                                  shuffle=True,
                                  drop_last=True) 

In [4]:
# 模型
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv3d(1, 8, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        self.conv2 = nn.Conv3d(8, 32, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.fc3 = nn.Linear(1000000, 32)
        self.fc4 = nn.Linear(32, num_classes)
        self.dropout = nn.Dropout(p=0.5)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool1(x)
        x = self.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.reshape(x.size(0), -1)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        res = self.fc4(x)
        return res
    
model = Net(2)

# TODO:define loss function and optimiter
#criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)
criterion = nn.MultiLabelSoftMarginLoss()
#optimizer = torch.optim.Adam(model.parameters())

In [5]:
# 训练
for epoch in range(NUM_EPOCHS):
    optimizer.zero_grad()
    for images,labels in train_voxel_loader:
        images = images.view(BATCH_SIZE,1,100,100,100)
        outputs = model(images.float())
        loss = criterion(outputs.t(),labels)
        loss.backward()
        optimizer.step()
    print(epoch)

NameError: name 'train_voxel_loader' is not defined

In [7]:
torch.save(model,'try_0_voxel_model.pkl')

/root/anaconda3/envs/myconda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/anaconda3/envs/myconda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv3d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/anaconda3/envs/myconda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type MaxPool3d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/root/anaconda3/envs/myconda/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + 

In [ ]:
# 测试
model = torch.load('try_0_voxel_model.pkl')
pred = torch.ones((0,2))
for images in test_voxel_loader:
    images = images.view(BATCH_SIZE,1,100,100,100)
    outputs = model(images.float())
    pred = torch.cat([pred,outputs])

In [18]:
# 输出
#pred = np.transpose(pred)
#np.savetxt('try000.csv', pred, delimiter = '\n') 
print(type(pred))
print(pred.shape)
print(pred.dtype)
print(pred)

<class 'torch.Tensor'>
torch.Size([32, 1])
torch.int64
tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]])


In [ ]:
print